# 1. Load data into pandas dataframes. I ignore information from holidays_events.csv and stores.csv cause it's not necessary for simple using the Prophet model.
###Credit to SERGIY GAVRYLOV for his inspiration

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from prophet import Prophet
from tqdm.notebook import tqdm
import logging
from joblib import Parallel, delayed

In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


df_train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
df_test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
df_oil =pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
df_holidays = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
#df_stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
df_sub =pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


# 2. Fix gaps in df_oil

In [3]:
# Let's create a continuous range from df_train.date.min() to df_test.date.max()
oil = np.arange(np.datetime64("2013-01-01"), np.datetime64("2017-09-01"), np.timedelta64(1, "D"))
df_full_oil = pd.DataFrame({'date':oil })
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_full_oil['date'] = pd.to_datetime(df_full_oil['date'])
df_full_oil = pd.merge(df_full_oil,df_oil, how='left')
df_full_oil['dcoilwtico'] = df_full_oil['dcoilwtico'].interpolate()
df_full_oil['dcoilwtico'] = df_full_oil['dcoilwtico'].bfill()
print(df_holidays.columns)
# Convert the 'date' column to datetime
df_holidays["date"] = pd.to_datetime(df_holidays["date"])

# Filter out the relevant holidays
df_holidays = df_holidays[df_holidays['transferred'] == False]

# Rename columns for compatibility with Prophet
df_holidays.columns = ['date', 'type', 'locale', 'locale_name', 'description', 'transferred']


Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')


# 3. Add oil cost info to df_train and df_test

In [4]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

holidays= df_holidays[['date', 'type']]
holidays.columns = ['ds', 'holiday']
holidays['ds']= pd.to_datetime(holidays['ds'])

merged_df = pd.merge(df_train, df_full_oil, left_on='date', right_on='date')
df_train['fuel_cost'] = merged_df['dcoilwtico']

merged_df = pd.merge(df_test, df_full_oil, left_on='date', right_on='date')
df_test['fuel_cost'] = merged_df['dcoilwtico']
print(df_train.columns)
print(df_test.columns)
print(holidays)

/tmp/ipykernel_18/2507942228.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holidays['ds']= pd.to_datetime(holidays['ds'])


Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion',
       'fuel_cost'],
      dtype='object')
Index(['id', 'date', 'store_nbr', 'family', 'onpromotion', 'fuel_cost'], dtype='object')
            ds     holiday
0   2012-03-02     Holiday
1   2012-04-01     Holiday
2   2012-04-12     Holiday
3   2012-04-14     Holiday
4   2012-04-21     Holiday
..         ...         ...
345 2017-12-22  Additional
346 2017-12-23  Additional
347 2017-12-24  Additional
348 2017-12-25     Holiday
349 2017-12-26  Additional

[338 rows x 2 columns]


# 4. In the loop we make prediction for each 'store' and 'family'

In [5]:
df_results = pd.DataFrame({
    'ds': pd.Series(dtype='datetime64[ns]'),
    'store_nbr': pd.Series(dtype='int'),
    'family': pd.Series(dtype='str'),
    'yhat': pd.Series(dtype='float')
})

logging.getLogger('cmdstanpy').setLevel(logging.CRITICAL)

grouped = df_train.groupby(['store_nbr', 'family']).count().index

def process_group(s, f):
    #Filtering and preparing training data
    df_simple = df_train[(df_train.store_nbr == s) & (df_train.family == f)][['date', 'sales', 'onpromotion', 'fuel_cost']]
    df_simple.columns = ['ds','y', 'onpromotion', 'fuel_cost']
    
    #initialize Prophet model
    m = Prophet(interval_width=0.95, daily_seasonality=True, holidays=holidays)
    m.add_regressor('onpromotion', prior_scale=0.5, mode='multiplicative')
    m.add_regressor('fuel_cost', prior_scale=0.5, mode='multiplicative')
    model = m.fit(df_simple)
    
    #Prepare future DF
    future = df_test[(df_test.store_nbr == s) & (df_test.family == f)][['date', 'onpromotion', 'fuel_cost']]
    future.columns = ['ds', 'onpromotion', 'fuel_cost']
    
    #Predictions
    forecast = m.predict(future)
    forecast['store_nbr'] = s
    forecast['family'] = f
    

    #Filter forecast results & return cols
    df_result = forecast[forecast['ds'] > '2017-08-15'][['ds', 'store_nbr', 'family', 'yhat']].copy()
    return df_result

results = Parallel(n_jobs =-1)(delayed(process_group)(s,f) for s, f in tqdm(grouped, desc="Processing", unit="group"))

df_results = pd.concat(results, ignore_index=True)

Processing:   0%|          | 0/1782 [00:00<?, ?group/s]

14:25:37 - cmdstanpy - INFO - Chain [1] start processing
14:25:38 - cmdstanpy - INFO - Chain [1] done processing
14:25:38 - cmdstanpy - INFO - Chain [1] start processing
14:25:38 - cmdstanpy - INFO - Chain [1] start processing
14:25:38 - cmdstanpy - INFO - Chain [1] done processing
14:25:39 - cmdstanpy - INFO - Chain [1] done processing
14:25:40 - cmdstanpy - INFO - Chain [1] start processing
14:25:40 - cmdstanpy - INFO - Chain [1] start processing
14:25:40 - cmdstanpy - INFO - Chain [1] start processing
14:25:41 - cmdstanpy - INFO - Chain [1] done processing
14:25:41 - cmdstanpy - INFO - Chain [1] done processing
14:25:41 - cmdstanpy - INFO - Chain [1] done processing
14:25:41 - cmdstanpy - INFO - Chain [1] start processing
14:25:41 - cmdstanpy - INFO - Chain [1] done processing
14:25:43 - cmdstanpy - INFO - Chain [1] start processing
14:25:43 - cmdstanpy - INFO - Chain [1] done processing
14:25:43 - cmdstanpy - INFO - Chain [1] start processing
14:25:43 - cmdstanpy - INFO - Chain [1]

# 5. Save results of predictions into file and submit

In [6]:
###### df_sub = pd.merge(df_test, df_results,  how='inner', left_on=['date','store_nbr', 'family'], right_on = ['ds','store_nbr', 'family'])[['id', 'yhat']]
df_sub.columns = ['id','sales']
#update negative predictions
df_sub.loc[df_sub[(df_sub['sales'] < 0)].index, 'sales'] = 0
df_sub.to_csv('/kaggle/working/submission.csv', index=False)